In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_html = requests.get(url).text
soup = BeautifulSoup(wiki_html, 'html.parser')

data = []
for tr in soup.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])

In [5]:
df=pd.DataFrame(data,columns=['PostalCode','Borough','Neighborhood2'])


In [6]:
# Find indexes of rows that have "Not assigned" in Borough column
indexNames = df[(df['Borough'] == "Not assigned")].index

# Drop rows that have "Not assigned" in Borough column
df.drop(indexNames,inplace=True)

# Drop the first row
df.dropna(inplace=True)

In [7]:
# Combine multiple rows into one row based on PostalCode and Borough
df=df.groupby(['PostalCode','Borough'])['Neighborhood2'].apply(', '.join).reset_index()

In [8]:
# Replace "Not assigned" in Neighborhood column with the value in Borough column
def custom_fx(data):
    if data['Neighborhood2']=='Not assigned':
        var=data['Borough']
    else:
        var=data['Neighborhood2']
    return var

# Apply the function
df['Neighborhood']=df.apply(custom_fx,axis='columns')

# Check that there is no more "Not assigned" in Neighborhood column
print("There are {} rows that have 'Not assigned' in Neighborhood column in the dataframe".format(
    len(df[df['Neighborhood']=='Not assigned'])
)
     )

# Delete Neighborhood2 column
df.drop(columns='Neighborhood2')

There are 0 rows that have 'Not assigned' in Neighborhood column in the dataframe


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [10]:
# Shape of the table
print("The shape of the dataframe is {}. The dataset has {} rows.".format
      (df.shape,df.shape[0]))

The shape of the dataframe is (103, 4). The dataset has 103 rows.


In [15]:
# Geospatial_Coordinates provided in the instruction
lonlat=pd.read_csv(r'http://cocl.us/Geospatial_data')

In [18]:
# Column names of lonlat
print("Column names of lonlat dataframe are: {}, {}, and {}.".format(lonlat.columns[0],lonlat.columns[1],lonlat.columns[2]))
print("Column names of trt dataframe are: {}, {}, and {}.".format(df.columns[0],df.columns[1],df.columns[2]))

# Change the name "Postal Code" in lonlat to "PostalCode"
lonlat.rename(columns={'Postal Code':'PostalCode'},inplace=True)

# Left join
df_geo=pd.merge(df,lonlat,how='left',on='PostalCode')

Column names of lonlat dataframe are: PostalCode, Latitude, and Longitude.
Column names of trt dataframe are: PostalCode, Borough, and Neighborhood2.


In [19]:
df_geo.to_csv(index=None,header=True)

'PostalCode,Borough,Neighborhood2,Neighborhood,Latitude,Longitude\nM1B,Scarborough,"Rouge, Malvern","Rouge, Malvern",43.806686299999996,-79.19435340000001\nM1C,Scarborough,"Highland Creek, Rouge Hill, Port Union","Highland Creek, Rouge Hill, Port Union",43.7845351,-79.16049709999999\nM1E,Scarborough,"Guildwood, Morningside, West Hill","Guildwood, Morningside, West Hill",43.7635726,-79.1887115\nM1G,Scarborough,Woburn,Woburn,43.7709921,-79.21691740000001\nM1H,Scarborough,Cedarbrae,Cedarbrae,43.773136,-79.23947609999999\nM1J,Scarborough,Scarborough Village,Scarborough Village,43.7447342,-79.23947609999999\nM1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park","East Birchmount Park, Ionview, Kennedy Park",43.7279292,-79.26202940000002\nM1L,Scarborough,"Clairlea, Golden Mile, Oakridge","Clairlea, Golden Mile, Oakridge",43.711111700000004,-79.2845772\nM1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West","Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.2394

In [21]:
df_geo

,PostalCode,Borough,Neighborhood2,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern","Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union","Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill","Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park","East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge","Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West","Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West","Birch Cliff, Cliffside West",43.692657,-79.264848
